# Guiding questions

How does graduate student gender diversity at Columbia compare to its peer institutions?

In [1]:
from collections import defaultdict
import math
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# https://stackoverflow.com/questions/19798112/convert-pandas-dataframe-to-a-nested-dict
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.ix[:,1:]) for k,g in grouped}
    return d

def get_dataframe():
    df = pd.read_csv(
        '../../data/GSS_Export_Broad_Sex_RaceEthnicity.csv', 
        thousands=',',
        header=4, # starting at 4th row
        skipfooter=4, # ignore last 4 lines of data (footer)
        engine='python', # default pandas engine does not support skipfooter
        index_col=False, # don't use first column (year) as dataframe index
        names = ['Year', 'Institution Name', 'Broad Fields', 'Sex', 'Race and Ethnicity', 'Total']
    )

    # Delete some aggregate values we don't need for this analysis
    df = df[ df['Institution Name'] != 'Total for selected values' ]
    df = df[ df['Broad Fields'] != 'Total for selected values' ]

    # Rename all instances of 'Columbia Univ in the City of New York'
    # to 'Columbia University in the City of New York'
    df['Institution Name'] = df['Institution Name'].str.replace(
        'Columbia Univ in the City of New York', 
        'Columbia University in the City of New York'
    )    
    return df

In [24]:
try:
    TOTALS = pickle.load( open('../../data/GSS_Export_Broad_Sex_RaceEthnicity.p', 'rb') )
except FileNotFoundError:
    TOTALS = recur_dictify( get_dataframe() )
    with open('../../data/GSS_Export_Broad_Sex_RaceEthnicity.p', 'wb') as f:
        pickle.dump(totals, f)

In [101]:
years = sorted(TOTALS.keys())
institutions = sorted(TOTALS[ max(years) ].keys())
fields = sorted(TOTALS[ max(years) ]['Columbia University in the City of New York'].keys())

print(f'Data set describes years {min(years)} - {max(years)} for the following institutions: {institutions}.')

Data set describes years 1994 - 2016 for the following institutions: ['Brown University', 'Columbia University in the City of New York', 'Cornell University', 'Dartmouth College', 'Duke University', 'Harvard University', 'Massachusetts Institute of Technology', 'Princeton University', 'Stanford University', 'University of Pennsylvania', 'Yale University'].


In [111]:
# Convenient function for making subplots for every institution
def subplots():
    cols = 2
    rows = math.ceil(len(institutions) / 2)
    return plt.subplots(rows, cols, figsize=(7 * cols, 7 * rows))

def compare_by_field(field):
    fig, axArr = subplots()
    for i, inst in enumerate(institutions):
        ax = axArr[i // 2, i % 2]
        plot_field(ax, inst, field)
    fig.suptitle(f'{field}')
    fig.tight_layout()

In [112]:
def plot_field(ax, inst, field):
    proportions = []
    for year in years:
        field_stats = TOTALS[year][inst].get(field, None)
        if field_stats is None:
            proportions.append(None)
        else:
            proportions.append(
                field_stats['Female']['Total for selected values'] /
                    field_stats['Total for selected values']['Total for selected values']
            )
    ax.set_title(inst)
    ax.set_ylim(top=1, bottom=0)
    ax.set_xlim(left=min(years), right=max(years))
    ax.plot(years, proportions)

In [ ]:
compare_by_field('Computer sciences')